12


In [37]:
from ortools.linear_solver import pywraplp as mip
import numpy.linalg as la
import numpy as np
import networkx as nx

# graph_3d = {}
# graph_3d[0, 0, 1, 0] = 0.9
# graph_3d[0, 0, 1, 1] = 0.9
# graph_3d[0, 1, 1, 0] = 0.9
# graph_3d[0, 1, 1, 1] = 0.9
# graph_3d[0, 0, 2, 0] = 0.9
# graph_3d[0, 0, 2, 1] = 0.9
# graph_3d[0, 1, 2, 0] = 0.9
# graph_3d[0, 1, 2, 1] = 0.9
# graph_3d[1, 0, 2, 0] = 0.9
# graph_3d[1, 0, 2, 1] = 0.9
# graph_3d[1, 1, 2, 0] = 0.9
# graph_3d[1, 1, 2, 1] = 0.9
# n_frames = 3

n_frames = 4
graph_3d = {}
for t1 in range(n_frames - 1):
    for pid1 in range(2):
        for t2 in range(t1 + 1, n_frames):
            for pid2 in range(2):
                ab = (t1, pid1, t2, pid2)
                graph_3d[ab] = 0.9
                
print(len(graph_3d))


solver = mip.Solver('t', mip.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

pids_per_frame = {}
Tau = {}
costs = {}
for (tA, pidA, tB, pidB), score in graph_3d.items():
    Tau[tA, pidA, tB, pidB] = solver.BoolVar('t[%i,%i,%i,%i]' % (tA, pidA, tB, pidB))
    costs[tA, pidA, tB, pidB] = np.log(score / (1-score))
    if tA not in pids_per_frame:
        pids_per_frame[tA] = set()
    pids_per_frame[tA].add(pidA)
    if tB not in pids_per_frame:
        pids_per_frame[tB] = set()
    pids_per_frame[tB].add(pidB)

Sum = solver.Sum(Tau[edge] * costs[edge] for edge in graph_3d.keys())

for t1 in range(n_frames - 1):
    for t2 in range(t1 + 1, n_frames):
        for pid1 in pids_per_frame[t1]:
            solver.Add(
                solver.Sum(Tau[t1, pid1, t2, pid2] for pid2 in pids_per_frame[t2]\
                           if (t1, pid1, t2, pid2) in Tau) <= 1
            )
        
        for pid2 in pids_per_frame[t2]:
            solver.Add(
                solver.Sum(Tau[t1, pid1, t2, pid2] for pid1 in pids_per_frame[t1]\
                           if (t1, pid1, t2, pid2) in Tau) <= 1
            )

# -- transitivity --
for t1 in range(n_frames - 2):
    t2 = t1 + 1
    t3 = t1 + 2
    for pid1 in pids_per_frame[t1]:
        for pid2 in pids_per_frame[t2]:
            for pid3 in pids_per_frame[t3]:
                ab = (t1, pid1, t2, pid2)
                bc = (t2, pid2, t3, pid3)
                ac = (t1, pid1, t3, pid3)
                if not ab in Tau or not bc in Tau or not ac in Tau:
                    continue
                solver.Add(Tau[ab] + Tau[bc] - 1 <= Tau[ac])
                solver.Add(Tau[ab] + Tau[ac] - 1 <= Tau[bc])
                solver.Add(Tau[bc] + Tau[ac] - 1 <= Tau[ab])
            
solver.Maximize(Sum)
RESULT = solver.Solve()

node_lookup = {}  # t, pid -> node number
reverse_node_lookup = {}  # node number -> t pid
G = nx.Graph()
nid = 1
for t in range(n_frames):
    for pid in pids_per_frame[t]:
        node_lookup[t, pid] = nid
        reverse_node_lookup[nid] = (t, pid)
        G.add_node(nid, key=(t, pid))
        nid += 1

for (t1, pid1, t2, pid2), v in Tau.items():
    assert t1 < t2
    print(str((t1, pid1)) + '--' + str((t2, pid2)) + ', ', v.solution_value())
    if v.solution_value() > 0:
        nid1 = node_lookup[t1, pid1]
        nid2 = node_lookup[t2, pid2]
        c = costs[t1, pid1, t2, pid2]
        G.add_edge(nid1, nid2, cost=c)
        
for global_pid, comp in enumerate(nx.connected_components(G)):
    print('\ncomponent ', global_pid)
    for nid in comp:
        node = G.nodes[nid]
        t, local_pid = node['key']
        print(t, local_pid)

24
(0, 0)--(1, 0),  0.0
(0, 0)--(1, 1),  1.0
(0, 0)--(2, 0),  0.0
(0, 0)--(2, 1),  1.0
(0, 0)--(3, 0),  1.0
(0, 0)--(3, 1),  0.0
(0, 1)--(1, 0),  1.0
(0, 1)--(1, 1),  0.0
(0, 1)--(2, 0),  1.0
(0, 1)--(2, 1),  0.0
(0, 1)--(3, 0),  0.0
(0, 1)--(3, 1),  1.0
(1, 0)--(2, 0),  1.0
(1, 0)--(2, 1),  0.0
(1, 0)--(3, 0),  1.0
(1, 0)--(3, 1),  0.0
(1, 1)--(2, 0),  0.0
(1, 1)--(2, 1),  1.0
(1, 1)--(3, 0),  0.0
(1, 1)--(3, 1),  1.0
(2, 0)--(3, 0),  1.0
(2, 0)--(3, 1),  0.0
(2, 1)--(3, 0),  0.0
(2, 1)--(3, 1),  1.0

component  0
0 0
0 1
1 0
1 1
2 0
2 1
3 0
3 1
